# Workbook for Accessing CMIP6 Data via Amazon Web Services
### Authors

Samantha Stevenson sstevenson@ucsb.edu

### Table of Contents

[Goals](#purpose)

[Import Packages](#import)

[Load and Query the CMIP6 AWS Catalog](#data_io)

[Plot a Time Series](#time_series)

<a id='purpose'></a> 
# Goals

This is the companion "workbook" for the tutorials in the "CMIP6 AWS" Climate DataLab repository. It does not contain any code! You can use this as a space to create your own workflow, based on the steps in the tutorials. We have provided an overall structure for the workflow, but you are encouraged to customize your code using snippets from the tutorials in this or other repositories as you like.

Happy coding!

<a id='import'></a> 
# Import Packages

Import all packages necessary for this tutorial: the main ones you'll need are matplotlib, xarray, intake, and s3fs. 

_Note: make sure you've installed the intake-esm plugin for the intake package, or that you're working on a server where this was done for you!_

In [1]:
# Import necessary packages
import xarray as xr
import matplotlib.pyplot as plt
import intake
import s3fs

<a id='data_io'></a> 
# Load and Query the CMIP6 AWS Catalog

Now use the steps laid out in the [Accessing CMIP6 Data via AWS](https://github.com/climate-datalab/CMIP6_AWS/blob/main/CMIP6_timeseries_map.ipynb) tutorial to load your data using intake!

In [2]:
# Open the CMIP6 data catalog, store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

Once the data is loaded, you can query it using various search terms, listed below:

- _activity_id_: which project do you want? CMIP = historical data, ScenarioMIP = future projections

- _source_id_: which model do you want? 

- _experiment_id_: what experimental configuration do you want? Here we want historical and the four main SSPs

- _member_id_: which ensemble member do you want? Here we want r10i1p1f1

- _table_id_: which part of the Earth system and time resolution do you want? Here we want monthly atmosphere data

- _variable_id_: which climate variable do you want? Here we want surface air temperature


### How do I figure out what I'm looking at??

Try the following to get a sense for what's available in the catalog:

1) **List all unique models participating in the "CMIP" activity** ("activity_id=CMIP")
  
  (_see the section "Example: finding all the unique model names contributing to a given activity" in Tutorial 1_)

In [3]:
# Search through catalog, find all historical simulations
cmip = catalog.search(activity_id = "CMIP", experiment_id = "historical")

# Convert to a data frame
cmip_df = cmip.df

# Get unique model names in the set of search results
cmip_model_names = cmip_df.source_id.unique()

# Print list of model names
print(f"List of model names:\n\n{cmip_model_names}")

List of model names:

['GFDL-CM4' 'GFDL-ESM4' 'IPSL-CM6A-LR' 'GISS-E2-1-G' 'CNRM-CM6-1'
 'BCC-CSM2-MR' 'BCC-ESM1' 'CNRM-ESM2-1' 'MIROC6' 'AWI-CM-1-1-MR'
 'MRI-ESM2-0' 'CESM2-WACCM' 'CanESM5' 'CESM2' 'SAM0-UNICON' 'GISS-E2-1-H'
 'UKESM1-0-LL' 'EC-Earth3' 'CanESM5-CanOE' 'INM-CM4-8' 'EC-Earth3-Veg'
 'INM-CM5-0' 'HadGEM3-GC31-LL' 'MPI-ESM-1-2-HAM' 'NESM3' 'CAMS-CSM1-0'
 'MPI-ESM1-2-LR' 'MPI-ESM1-2-HR' 'E3SM-1-0' 'MCM-UA-1-0' 'NorESM2-LM'
 'GISS-E2-1-G-CC' 'FGOALS-g3' 'FGOALS-f3-L' 'MIROC-ES2L' 'KACE-1-0-G'
 'NorCPM1' 'CNRM-CM6-1-HR' 'KIOST-ESM' 'NorESM2-MM' 'ACCESS-CM2'
 'FIO-ESM-2-0' 'ACCESS-ESM1-5' 'CESM2-FV2' 'CESM2-WACCM-FV2' 'GISS-E2-2-H'
 'E3SM-1-1' 'HadGEM3-GC31-MM' 'IITM-ESM' 'CIESM' 'E3SM-1-1-ECA'
 'AWI-ESM-1-1-LR' 'EC-Earth3-Veg-LR' 'TaiESM1' 'CAS-ESM2-0' 'CMCC-CM2-SR5'
 'EC-Earth3-AerChem' 'IPSL-CM5A2-INCA' 'CMCC-CM2-HR4' 'EC-Earth3P-VHR'
 'EC-Earth3-CC' 'CMCC-ESM2' 'MIROC-ES2H' 'ICON-ESM-LR' 'IPSL-CM6A-LR-INCA']


2) **List all unique ensemble members** associated with the "historical" simulations ("experiment_id=historical") run with CanESM5 ("source_id=CanESM5")

   _(see the section "Example: finding all the historical simulations with a given model¶") in Tutorial 1)_

In [4]:
# Search through catalog, find all historical simulations with CanESM5
historical = catalog.search(activity_id="CMIP", experiment_id="historical", source_id="CanESM5")

# Convert to a data frame
historical_df = historical.df

# Print all unique ensemble members ("member_id")
print(f"List of historical ensemble names:\n\n{historical_df.member_id.unique()}")

List of historical ensemble names:

['r24i1p1f1' 'r25i1p1f1' 'r14i1p1f1' 'r2i1p1f1' 'r17i1p1f1' 'r10i1p1f1'
 'r13i1p1f1' 'r7i1p1f1' 'r6i1p1f1' 'r5i1p1f1' 'r3i1p1f1' 'r22i1p1f1'
 'r23i1p1f1' 'r8i1p1f1' 'r11i1p1f1' 'r12i1p1f1' 'r15i1p1f1' 'r19i1p1f1'
 'r16i1p1f1' 'r1i1p1f1' 'r9i1p1f1' 'r18i1p1f1' 'r4i1p1f1' 'r21i1p1f1'
 'r20i1p1f1' 'r11i1p2f1' 'r10i1p2f1' 'r7i1p2f1' 'r9i1p2f1' 'r8i1p2f1'
 'r4i1p2f1' 'r40i1p2f1' 'r3i1p2f1' 'r6i1p2f1' 'r24i1p2f1' 'r13i1p2f1'
 'r12i1p2f1' 'r5i1p2f1' 'r31i1p2f1' 'r30i1p2f1' 'r32i1p2f1' 'r29i1p2f1'
 'r28i1p2f1' 'r2i1p2f1' 'r22i1p2f1' 'r23i1p2f1' 'r26i1p2f1' 'r27i1p2f1'
 'r25i1p2f1' 'r37i1p2f1' 'r38i1p2f1' 'r39i1p2f1' 'r35i1p2f1' 'r34i1p2f1'
 'r36i1p2f1' 'r33i1p2f1' 'r1i1p2f1' 'r18i1p2f1' 'r19i1p2f1' 'r14i1p2f1'
 'r15i1p2f1' 'r17i1p2f1' 'r16i1p2f1' 'r21i1p2f1' 'r20i1p2f1']


### **Find a specific file**

Let's do an example of pulling the data we used in previous tutorials, this time from the cloud. The example data file from the [Time Series Plots](https://github.com/climate-datalab/Time-Series-Plots) and [Map Plots](https://github.com/climate-datalab/Map-Plots) repositories is:

`tas_Amon_CanESM5_historical_r10i1p1f1_gn_185001-201412.nc`

We can break this down to extract the fields we'll need to search the data catalog properly. If you need more detail on how to do this, also refer to the [filename decoder](http://climate-datalab.org/filename-decoder/) on the Climate DataLab website!

#### **Characteristics of this file (corresponding fields in the CMIP6 catalog are in parentheses)**:
- _Variable ("variable_id")_: This is a surface air temperature, or "tas", variable.
- _Realm ("table_id")_: Surface air temperature is generated by the atmosphere component of a climate model ("A"), and the information in this particular file is averaged monthly ("mon").
- _Model ("source_id")_: The name of the model is "CanESM5", which is short for the Canadian Earth System Model version 5.
- _Experiment ("experiment_id")_: The name of the model experiment being run. The file above is a _historical_ simulation: since we're also interested in the future projection information, we'll further specify that we'd also like the associated SSPs below.
- _Ensemble member ("member_id")_: The name of this ensemble member is "r10i1p1f1".

In [5]:
# Specify search terms to query catalog for CanESM5 data
# activity_id: which project do you want? CMIP = historical data, ScenarioMIP = future projections
activity_ids = ['ScenarioMIP', 'CMIP'] 

# source_id: which model do you want? 
model = ["CanESM5"]

# experiment_id: what experimental configuration do you want? Here we want historical and the four main SSPs
experiment_ids = ['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']

# member_id: which ensemble member do you want? Here we want r10i1p1f1
member_id = 'r10i1p1f1'

# table_id: which part of the Earth system and time resolution do you want? Here we want monthly atmosphere data
realm = "Amon"

# variable_id: which climate variable do you want? Here we want surface air temperature
variable_id = "tas"

In [6]:
# Search through catalog, store results in "res" variable
search_result = catalog.search(activity_id = activity_ids, source_id = model, experiment_id = experiment_ids, member_id = member_id, table_id = realm, variable_id = variable_id)

# Display data frame associated with results
search_result_df = search_result.df

# Extract data for the historical period, store as a separate xarray Dataset
search_result_xr_hist = xr.open_zarr(search_result_df[search_result_df["experiment_id"] == "historical"]["zstore"][0], storage_options = {'anon':True})

# Extract data for an SSP 
search_result_df_fut = xr.open_zarr(search_result_df["zstore"][4], storage_options = {'anon':True})

<a id='time_series'></a> 
## **Plot a Time Series**

Once the data have been loaded in, you can use it to generate a time series, following the exact same steps used in previous tutorials!

In [7]:
# Concatenate historical and future projection data

# Convert time to datetime64 format


In [8]:
# Define min/max bounds for region of interest 


# Define logical mask: True when lat/lon inside the valid ranges, False elsewhere

# Find points where the mask value is True, drop all other points

# Average over lat, lon dimensions to get a time series


In [9]:
# Plot the resulting time series